In [2]:
import configparser
import csv
from collections import Counter
import nbimporter
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import math

### Lendo arquivo de configuração

In [3]:
import ast

In [4]:
config = configparser.ConfigParser()
config.read('BUSCA.cfg')

modelo_path = config.get('DEFAULT', 'MODELO')

modelo_vetorial = {}
with open(modelo_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    
    for row in reader:
        key_str = row[0].replace("(", "").replace(")", "")
        key_tuple = tuple(ast.literal_eval(key_str))
        value = float(row[1])
        
        modelo_vetorial[key_tuple] = value

consultas_path = config.get('DEFAULT', 'CONSULTAS')
results_path = config.get('DEFAULT', 'RESULTADOS')

In [5]:
q1 = 'cystic infection in the respiratory tract'
q2 = 'cystic fibrosis in the respiratory tract'

In [6]:
modelo_vetorial

{('00001 ', 'SIGNIFICANCE'): 0.04154799294201786,
 ('00074 ', 'SIGNIFICANCE'): 0.08781734871835592,
 ('00078 ', 'SIGNIFICANCE'): 0.07576398712956198,
 ('00121 ', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00147 ', 'SIGNIFICANCE'): 0.04067329835376485,
 ('00157 ', 'SIGNIFICANCE'): 0.14310975346695037,
 ('00179', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00185', 'SIGNIFICANCE'): 0.028621950693390078,
 ('00195', 'SIGNIFICANCE'): 0.0536661575501064,
 ('00205', 'SIGNIFICANCE'): 0.03018721362193485,
 ('00223', 'SIGNIFICANCE'): 0.058544899145570616,
 ('00258', 'SIGNIFICANCE'): 0.019514966381856874,
 ('00319', 'SIGNIFICANCE'): 0.052215720859562983,
 ('00355', 'SIGNIFICANCE'): 0.04246113564404023,
 ('00402 ', 'SIGNIFICANCE'): 0.12879877812025536,
 ('00411 ', 'SIGNIFICANCE'): 0.031933581352129425,
 ('00412 ', 'SIGNIFICANCE'): 0.08221198603420554,
 ('00485 ', 'SIGNIFICANCE'): 0.0633436613706174,
 ('00526 ', 'SIGNIFICANCE'): 0.0330253277231424,
 ('00555 ', 'SIGNIFICANCE'): 0.06899934542156537,
 ('0

In [7]:
%store -r li
li

{'SIGNIFICANCE': ['00001 ',
  '00074 ',
  '00078 ',
  '00121 ',
  '00147 ',
  '00157 ',
  '00179',
  '00185',
  '00195',
  '00205',
  '00223',
  '00258',
  '00319',
  '00355',
  '00402 ',
  '00411 ',
  '00412 ',
  '00485 ',
  '00526 ',
  '00555 ',
  '00593 ',
  '00631 ',
  '00809 ',
  '00858 ',
  '00923 ',
  '01151 '],
 'PSEUDOMONAS': ['00001 ',
  '00001 ',
  '00001 ',
  '00007 ',
  '00008 ',
  '00018 ',
  '00018 ',
  '00061 ',
  '00061 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00079 ',
  '00080 ',
  '00081 ',
  '00081 ',
  '00081 ',
  '00081 ',
  '00082 ',
  '00082 ',
  '00082 ',
  '00123 ',
  '00123 ',
  '00123 ',
  '00135 ',
  '00148 ',
  '00152 ',
  '00159 ',
  '00160 ',
  '00161 ',
  '00176',
  '00177',
  '00177',
  '00178',
  '00188',
  '00188',
  '00188',
  '00188',
  '00200',
  '00200',
  '00250',
  '00260',
  '00265',
  '00271',
  '00282',
  '00282',
  '00384 ',
  '00384 ',
  '00394 ',
  '00427 ',
  '00427 ',
  '00447 ',
  '00458 ',
  '00459 ',
  '00460

In [8]:
%store -r n_words
%store -r li
N = len(n_words.keys())

In [10]:
def vms(tfidf, query):
    query_tokens = [token.upper() for token in word_tokenize(query) if token not in nltk.corpus.stopwords.words('english')]
    query_weights = {}
    docs_weights = {}
    D = {}
    dot_QD = {}

    tf_query = Counter(query_tokens)

    # compute each token weight with respect to the QUERY
    for token in query_tokens:
        tf = tf_query[token] / len(query_tokens)
        idf = N / len(set(li[token]))
        query_weights[token] = tf * idf

    # compute each token weight with respect to EACH DOCUMENT
    for (doc_id, token), weight in tfidf.items():
        if token in query_tokens:
            if doc_id not in docs_weights:
                docs_weights[doc_id] = []
            docs_weights[doc_id].append((token, weight))
    print(f"dict com token da query = (peso): {query_weights}\n\n")
    print(f"dict com doc_id = (token, peso): {docs_weights}\n\n")

    for doc_id, token_weight_list in docs_weights.items():
    # Extract common tokens present in both qw and the current document
        common_tokens = set(query_weights.keys()) & set(token for token, _ in token_weight_list)
        
        # Create a vector for qw containing only weights associated with common tokens
        qw_vector = np.array([query_weights[token] for token in common_tokens])

        
        # Create document vector containing weights for common tokens
        doc_vector = np.array([weight for token, weight in token_weight_list if token in common_tokens])
        
        # Compute the dot product between qw_vector and doc_vector
        dot_QD[doc_id] = np.dot(qw_vector, doc_vector)
        print(common_tokens)
        print(qw_vector)
        print(doc_vector)
    # return dot_QD

In [14]:
modelo_vetorial

{('00001 ', 'SIGNIFICANCE'): 0.04154799294201786,
 ('00074 ', 'SIGNIFICANCE'): 0.08781734871835592,
 ('00078 ', 'SIGNIFICANCE'): 0.07576398712956198,
 ('00121 ', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00147 ', 'SIGNIFICANCE'): 0.04067329835376485,
 ('00157 ', 'SIGNIFICANCE'): 0.14310975346695037,
 ('00179', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00185', 'SIGNIFICANCE'): 0.028621950693390078,
 ('00195', 'SIGNIFICANCE'): 0.0536661575501064,
 ('00205', 'SIGNIFICANCE'): 0.03018721362193485,
 ('00223', 'SIGNIFICANCE'): 0.058544899145570616,
 ('00258', 'SIGNIFICANCE'): 0.019514966381856874,
 ('00319', 'SIGNIFICANCE'): 0.052215720859562983,
 ('00355', 'SIGNIFICANCE'): 0.04246113564404023,
 ('00402 ', 'SIGNIFICANCE'): 0.12879877812025536,
 ('00411 ', 'SIGNIFICANCE'): 0.031933581352129425,
 ('00412 ', 'SIGNIFICANCE'): 0.08221198603420554,
 ('00485 ', 'SIGNIFICANCE'): 0.0633436613706174,
 ('00526 ', 'SIGNIFICANCE'): 0.0330253277231424,
 ('00555 ', 'SIGNIFICANCE'): 0.06899934542156537,
 ('0

In [55]:
def vms(tfidf, query):
    query_tokens = [token.upper() for token in word_tokenize(query) if token not in nltk.corpus.stopwords.words('english')]
    query_weights = {}
    docs_weights = {}
    D = {}
    dot_QD = {}
    q_d_vec_diffs = {}

    tf_query = Counter(query_tokens)

    # compute each token weight with respect to the QUERY
    for token in query_tokens:
        tf = tf_query[token] / len(query_tokens)
        idf = N / len(set(li[token]))
        query_weights[token] = tf * idf

    # compute each token weight with respect to EACH DOCUMENT
    for (doc_id, token), weight in tfidf.items():
        if token in query_tokens:
            if doc_id not in docs_weights:
                docs_weights[doc_id] = []
            docs_weights[doc_id].append((token, weight))
            
    for doc_id, token_weight_list in docs_weights.items():
    # sorting docs_weight
        docs_weights[doc_id] = sorted(token_weight_list)

    for doc_id, token_weight_list in docs_weights.items():
        common_QD_tokens = sorted(token for (token, _) in token_weight_list if token in query_weights.keys())
        qw_vector = np.array([query_weights[token] for token in common_QD_tokens])

        # create document vector containing weights for common tokens
        doc_vector = np.array([weight for token, weight in token_weight_list if token in common_QD_tokens])
        
        
        sum_of_squares = sum(dw**2 for _, dw in token_weight_list)
        D[doc_id] = math.sqrt(sum_of_squares)

        # compute scalar products of Query weights vector with each document weights vector
        dot_QD[doc_id] = np.dot(qw_vector, doc_vector)
        q_d_vec_diffs[doc_id] = qw_vector - doc_vector
    Q = math.sqrt(sum([qw**2 for qw in query_weights.values()]))
    return dot_QD, Q, D, q_d_vec_diffs

In [56]:
vms(modelo_vetorial, q1)[3]

{'00001 ': array([0.28948112, 2.50594135, 2.87038389, 5.63504415]),
 '00006 ': array([0.28980576, 2.52087324, 2.90744656, 5.68209256]),
 '00016 ': array([0.28676787, 2.51454805]),
 '00018 ': array([2.56525981]),
 '00048 ': array([2.53311457, 5.70381136]),
 '00057 ': array([0.27842932, 2.48397633]),
 '00058 ': array([0.28958035, 2.53408701]),
 '00061 ': array([0.28974663, 2.56021785]),
 '00062 ': array([0.28972146, 2.38848749]),
 '00079 ': array([2.54836876]),
 '00081 ': array([2.54970178]),
 '00104 ': array([0.29017849, 2.56683909]),
 '00111 ': array([0.28834989, 2.43268657]),
 '00123 ': array([0.2893682 , 2.55441588]),
 '00133 ': array([0.28096714, 2.54234053, 2.90886501]),
 '00146 ': array([0.28502765, 2.53455864]),
 '00159 ': array([0.28884575, 2.5464057 , 2.91316269, 5.68934879]),
 '00160 ': array([0.28643321, 2.4735007 , 2.76014619, 5.68790994]),
 '00176': array([0.2891409 , 2.55093093, 2.91794676, 5.69542181]),
 '00179': array([0.28977091, 2.5605901 , 2.92815841, 5.70838478]),
 '

In [59]:
def cosine_similarity(dot_products, qw_vector_mag, dw_vector_mag):
    cs_docs = {}

    for doc_id, dp in dot_products.items():
        cs_docs[doc_id] = dp / qw_vector_mag * dw_vector_mag[doc_id]

    return cs_docs
    

In [52]:
vms(modelo_vetorial, q1)[1]

{'CYSTIC': 0.2911184210526316, 'INFECTION': 2.58125, 'RESPIRATORY': 2.95, 'TRACT': 5.736111111111111}


6.9536373342725115

In [51]:
vms(modelo_vetorial, q1)[2]

{'CYSTIC': 0.2911184210526316, 'INFECTION': 2.58125, 'RESPIRATORY': 2.95, 'TRACT': 5.736111111111111}


{'00001 ': 0.14908832939393038,
 '00006 ': 0.09151980879564295,
 '00016 ': 0.06684367573620285,
 '00018 ': 0.015990192731829427,
 '00048 ': 0.05796803514819375,
 '00057 ': 0.098097811575507,
 '00058 ': 0.04718806574627336,
 '00061 ': 0.021076834691595495,
 '00062 ': 0.1927675687700371,
 '00079 ': 0.03288124139221263,
 '00081 ': 0.031548218092528335,
 '00104 ': 0.014441534881278393,
 '00111 ': 0.14858922104840555,
 '00123 ': 0.026891133916863215,
 '00133 ': 0.057524628294739276,
 '00146 ': 0.04708694955624336,
 '00159 ': 0.06901443763913474,
 '00160 ': 0.22360609164102577,
 '00176': 0.06005152366002634,
 '00179': 0.0409200647948852,
 '00187': 0.04534295898791403,
 '00188': 0.119201792675214,
 '00200': 0.10301999237718125,
 '00202': 0.040280624433723586,
 '00212': 0.06297633192765838,
 '00213': 0.029993957061116662,
 '00238': 0.02973550489965964,
 '00247': 0.03335097341210138,
 '00253': 0.02427162348259967,
 '00262': 0.08998187118334999,
 '00270': 0.0409573357692473,
 '00271': 0.13448791

In [46]:
vms(modelo_vetorial, q1)[1]

6.9536373342725115